In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Creatin de l'API

In [ ]:
!pip install flask_ngrok
!pip install flask
!pip install pyngrok

In [ ]:
!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
 | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
 && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
 | sudo tee /etc/apt/sources.list.d/ngrok.list \
 && sudo apt update \
 && sudo apt install ngrok

In [ ]:
!ngrok config add-authtoken ngrok_token

In [ ]:
pip install Flask joblib pandas scikit-learn


In [ ]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from flask_ngrok import run_with_ngrok

from pyngrok import ngrok

app = Flask(__name__)
run_with_ngrok(app)

# Chargement des modèles
logistic_model = joblib.load("/content/drive/MyDrive/Mes projets/Projet Systeme d'evaluation de risque de credit/Sauvegarde de model de production/modele_logistique_Drucky.pkl")
rf_model = joblib.load("/content/drive/MyDrive/Mes projets/Projet Systeme d'evaluation de risque de credit/Sauvegarde de model de production/modele_foret_aleatoire_Drucky.pkl")

# Initialisation du scaler et de l'encoder
scaler = joblib.load("/content/drive/MyDrive/Mes projets/Projet Systeme d'evaluation de risque de credit/Sauvegarde de model de production/scaler.pkl")
label_encoder = joblib.load( "/content/drive/MyDrive/Mes projets/Projet Systeme d'evaluation de risque de credit/Sauvegarde de model de production/label_encoder.pkl")


if not isinstance(scaler, StandardScaler):
    raise ValueError("L'objet chargé n'est pas un StandardScaler")


# Définir l'authtoken
ngrok.set_auth_token("ngrok_token")
# url creer par ngrok pour acceder a l'api sur le port 5000
http_tunnel = ngrok.connect(5000)
print(http_tunnel.public_url)


@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()

    # Convertir les données recupere depuis l'application utilisant l'api en DataFrame
    df = pd.DataFrame([data])
    X = df.drop(columns=['Client_ID','Nom_Client', 'Prenom_Client', 'predictionDate'])

    # Normaliser les données
    df_scaled = scaler.transform(X)  # Normalisation avec le scaler chargé
    # Prédictions
    logistic_prediction = logistic_model.predict(df_scaled)
    rf_prediction = rf_model.predict(df_scaled)

    # Retourner les résultats sous forme JSON
    return jsonify({
        'logistic_prediction': label_encoder.inverse_transform([logistic_prediction[0]])[0],
        'rf_prediction': label_encoder.inverse_transform([rf_prediction[0]])[0]
    })

if __name__ == '__main__':
    app.run()

Fonction ajoutees pour les commentaires d'apres predictions. Pour la future version pro

In [ ]:
# fonction a editer pour la version pro
def generate_comment(prediction, input_data):
    comments = []

    if prediction == 1:  # Risque élevé
        comments.append("Le client présente un risque élevé en raison de plusieurs facteurs.")

        if input_data['Taux_Endettement'] > 0.4:
            comments.append("Le taux d'endettement est supérieur à 40%, ce qui est préoccupant.")

        if input_data['Revenu_Annuel'] < 30000:
            comments.append("Le revenu annuel est relativement bas, augmentant le risque.")

    else:  # Risque faible
        comments.append("Le client présente un risque faible.")

        if input_data['Taux_Endettement'] < 0.2:
            comments.append("Le taux d'endettement est inférieur à 20%, ce qui est positif.")

        if input_data['Revenu_Annuel'] > 50000:
            comments.append("Le revenu annuel est suffisant, ce qui réduit le risque.")

    return " ".join(comments)


Api avec commentaires pour la future version pro

In [ ]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from flask_ngrok import run_with_ngrok

from pyngrok import ngrok

app = Flask(__name__)
run_with_ngrok(app)

# Chargement des modèles
logistic_model = joblib.load("/content/drive/MyDrive/Mes projets/Projet Systeme d'evaluation de risque de credit/Sauvegarde de model de production/modele_logistique_Drucky.pkl")
rf_model = joblib.load("/content/drive/MyDrive/Mes projets/Projet Systeme d'evaluation de risque de credit/Sauvegarde de model de production/modele_foret_aleatoire_Drucky.pkl")

# Initialisation du scaler
scaler = joblib.load("/content/drive/MyDrive/Mes projets/Projet Systeme d'evaluation de risque de credit/Sauvegarde de model de production/scaler.pkl")


# Définir l'authtoken
ngrok.set_auth_token("your_ngrok_token")
# url creer par ngrok pour acceder a l'api sur le port 5000
http_tunnel = ngrok.connect(5000)
print(http_tunnel.public_url)


@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()

    # Convertir les données recupere depuis l'application utilisant l'api en DataFrame
    df = pd.DataFrame([data])
    X = df.drop(columns=['Client_ID','Nom_Client', 'Prenom_Client', 'predictionDate'])

    # Normaliser les données
    df_scaled = scaler.transform(X)  # Normalisation avec le scaler chargé
    # Prédictions
    logistic_prediction = logistic_model.predict(df_scaled)
    rf_prediction = rf_model.predict(df_scaled)

    # Générer des commentaires
    logistic_comment = generate_comment(logistic_prediction[0], data)
    rf_comment = generate_comment(rf_prediction[0], data)

    return jsonify({
        'logistic_prediction': label_encoder.inverse_transform([logistic_prediction[0]])[0],
        'logistic_comment': logistic_comment,
        'rf_prediction': label_encoder.inverse_transform([rf_prediction[0]])[0],
        'rf_comment': rf_comment,
    })

if __name__ == '__main__':
    app.run()